In [2]:
import pandas as pd
import datetime
etfdata = pd.read_csv('NIFTYBEES.NS.csv')

In [3]:
l=[[]]
for i in list(etfdata['Date']):
    z = list(etfdata['Date'].loc[lambda x:x==i].index)
    etfdata.loc[z[0],'Weekday'] = datetime.datetime.strptime(i,'%d-%m-%y').weekday()

etfdata
#     if datetime.datetime.strptime(i,'%d-%m-%y').weekday() == 0:
#         l.append([0])
#     else:
#         l[len(l)-1].append(datetime.datetime.strptime(i,'%d-%m-%y').weekday())

# # print(l)
# for x in l:
#     if len(x) == 5:
#         continue
#     else:
#         print(x)

,Date,Open,High,Low,Close,Adj Close,Volume,Weekday
0,02-01-09,30.600000,30.945000,30.299999,30.556000,30.556000,1344830.0,4.0
1,05-01-09,30.700001,31.370001,30.700001,31.174999,31.174999,2429180.0,0.0
2,06-01-09,31.100000,31.480000,30.603001,31.124001,31.124001,1418730.0,1.0
3,07-01-09,31.379999,31.500000,28.900000,29.264999,29.264999,5724640.0,2.0
4,09-01-09,29.000000,29.500000,28.030001,28.855000,28.855000,1820800.0,4.0
...,...,...,...,...,...,...,...,...
3225,21-01-22,191.990005,191.990005,189.220001,190.639999,190.639999,3135380.0,4.0
3226,24-01-22,196.800003,196.800003,184.050003,185.789993,185.789993,9706128.0,0.0
3227,25-01-22,185.490005,187.380005,183.050003,187.070007,187.070007,5883754.0,1.0
3228,27-01-22,194.000000,194.000000,182.699997,185.330002,185.330002,5945584.0,3.0


Getting Dates list for every unique month

In [17]:
l={}
for i in list(etfdata['Date']):
    # z = list(etfdata['Date'].loc[lambda x:x==i].index)
    if (datetime.datetime.strptime(i,'%d-%m-%y').month,datetime.datetime.strptime(i,'%d-%m-%y').year) in l.keys():
        l[datetime.datetime.strptime(i,'%d-%m-%y').month,datetime.datetime.strptime(i,'%d-%m-%y').year].append(i)
    else:
        l[datetime.datetime.strptime(i,'%d-%m-%y').month,datetime.datetime.strptime(i,'%d-%m-%y').year] = [i]
    



Identifying Expiry Dates and adding to Database

In [82]:
my_columns = ['Date','Day','Buy Price','Qty','Investment Amount']
expirySIP = pd.DataFrame(columns=my_columns)
import math
SIPamount = 5000
expiry = {}
for i in l.keys():
    for y in l[i][::-1]:
        if datetime.datetime.strptime(y,'%d-%m-%y').weekday() == 3:
            z = list(etfdata['Date'].loc[lambda x:x==y].index)
            if math.isnan(etfdata.loc[z[0],'Close']):
                date = l[i][l[i].index(y)-1]
                price = etfdata.loc[z[0]-1,'Close']
                qty = math.floor(SIPamount/price)
                expirySIP = expirySIP.append(
                    pd.Series(
                        [
                            date,
                            datetime.datetime.strptime(y,'%d-%m-%y').weekday(),
                            price,
                            qty,
                            qty*price
                        ],index=my_columns
                    ),ignore_index=True
                )
            else:
                date = y
                price = etfdata.loc[z[0],'Close']
                qty = math.floor(SIPamount/price)
                expirySIP = expirySIP.append(
                    pd.Series(
                        [
                            date,
                            datetime.datetime.strptime(y,'%d-%m-%y').weekday(),
                            price,
                            qty,
                            qty*price
                        ],index=my_columns
                    ),ignore_index=True
                )
            break
expirySIP.to_csv('Expiry SIP.csv')

Normal 1st Trading Day SIP Plan to database

In [81]:
my_columns = ['Date','Day','Buy Price','Qty','Investment Amount']
normalSIP = pd.DataFrame(columns=my_columns)
for i in l.keys():
    for y in l[i]:
        date = datetime.datetime.strptime(y,'%d-%m-%y')
        z = list(etfdata['Date'].loc[lambda x:x==y].index)
        if math.isnan(etfdata.loc[z[0],'Close']):
            print(y)
            if math.isnan(etfdata.loc[z[0]+1,'Close']):
                continue
            # else:
                # print(y)
        else:
            date = y
            # print(date)
            price = etfdata.loc[z[0],'Close']
            qty = math.floor(SIPamount/price)
            normalSIP = normalSIP.append(
                    pd.Series(
                        [
                            date,
                            datetime.datetime.strptime(y,'%d-%m-%y').weekday(),
                            price,
                            qty,
                            qty*price
                        ],index=my_columns
                    ),ignore_index=True
                )
            break
normalSIP.to_csv('Normal SIP.csv')

SIP on the Month Low

In [90]:
my_columns = ['Date','Day','Buy Price','Qty','Investment Amount']
minSIP = pd.DataFrame(columns=my_columns)
for i in l.keys():
    min = 0
    mindate = ''
    for y in l[i]:
        # date = datetime.datetime.strptime(y,'%d-%m-%y')
        z = list(etfdata['Date'].loc[lambda x:x==y].index)
        price = etfdata.loc[z[0],'Close']
        
        if math.isnan(price):
            continue
        elif min == 0:
            min = price
            mindate = y
        elif price<min:
            min = price
            mindate = y
        else:
            continue
    qty = math.floor(SIPamount/min)
    minSIP = minSIP.append(
                    pd.Series(
                        [
                            mindate,
                            datetime.datetime.strptime(mindate,'%d-%m-%y').weekday(),
                            min,
                            qty,
                            qty*min
                        ],index=my_columns
                    ),ignore_index=True
                )

minSIP.to_csv('Min SIP.csv')
        


SIP on Maximum Number

In [91]:
my_columns = ['Date','Day','Buy Price','Qty','Investment Amount']
maxSIP = pd.DataFrame(columns=my_columns)
for i in l.keys():
    max = 0
    maxdate = ''
    for y in l[i]:
        # date = datetime.datetime.strptime(y,'%d-%m-%y')
        z = list(etfdata['Date'].loc[lambda x:x==y].index)
        price = etfdata.loc[z[0],'Close']
        
        if math.isnan(price):
            continue
        elif max == 0:
            max = price
            maxdate = y
        elif price>max:
            max = price
            maxdate = y
        else:
            continue
    qty = math.floor(SIPamount/max)
    maxSIP = maxSIP.append(
                    pd.Series(
                        [
                            maxdate,
                            datetime.datetime.strptime(maxdate,'%d-%m-%y').weekday(),
                            max,
                            qty,
                            qty*max
                        ],index=my_columns
                    ),ignore_index=True
                )

maxSIP.to_csv('Max SIP.csv')

Getting Dates list for every unique Week

In [106]:
w={}
for i in list(etfdata['Date']):
    # z = list(etfdata['Date'].loc[lambda x:x==i].index)
    if (datetime.datetime.strptime(i,'%d-%m-%y').isocalendar()[1],datetime.datetime.strptime(i,'%d-%m-%y').year) in w.keys():
        w[datetime.datetime.strptime(i,'%d-%m-%y').isocalendar()[1],datetime.datetime.strptime(i,'%d-%m-%y').year].append(i)
    else:
        w[datetime.datetime.strptime(i,'%d-%m-%y').isocalendar()[1],datetime.datetime.strptime(i,'%d-%m-%y').year] = [i]
# print(w)


SIP Every Week

In [128]:
my_columns = ['Date','Week','Buy Price','Qty','Investment Amount']
weekSIP = pd.DataFrame(columns=my_columns)
for z in w.keys():
    weeks = datetime.datetime(z[1],12,27).isocalendar()[1]
    WeeklySIP = (SIPamount*12)/weeks
    for y in w[z]:
        date = datetime.datetime.strptime(y,'%d-%m-%y')
        z = list(etfdata['Date'].loc[lambda x:x==y].index)
        if math.isnan(etfdata.loc[z[0],'Close']):
            continue
        else:
            price = etfdata.loc[z[0],'Close']
            date = y
            qty = math.floor(WeeklySIP/price)
            weekSIP = weekSIP.append(
                    pd.Series(
                        [
                            date,
                            datetime.datetime.strptime(date,'%d-%m-%y').isocalendar()[1],
                            price,
                            qty,
                            qty*price
                        ],index=my_columns
                    ),ignore_index=True
                )
            break
weekSIP.to_csv('Week SIP.csv')

Getting Trading Dates list for every unique Year

In [133]:
d={}
for i in list(etfdata['Date']):
    # z = list(etfdata['Date'].loc[lambda x:x==i].index)
    if (datetime.datetime.strptime(i,'%d-%m-%y').year) in d.keys():
        date = datetime.datetime.strptime(i,'%d-%m-%y')
        z = list(etfdata['Date'].loc[lambda x:x==i].index)
        if math.isnan(etfdata.loc[z[0],'Close']):
            continue
        else:
            d[datetime.datetime.strptime(i,'%d-%m-%y').year].append(i)
    else:
        date = datetime.datetime.strptime(i,'%d-%m-%y')
        z = list(etfdata['Date'].loc[lambda x:x==i].index)
        if math.isnan(etfdata.loc[z[0],'Close']):
            continue
        else:
            d[datetime.datetime.strptime(i,'%d-%m-%y').year] = [i]


SIP every day

In [145]:
my_columns = ['Date','Day','Buy Price','Qty','Investment Amount']
daySIP = pd.DataFrame(columns=my_columns)
for i in d.keys():
    totalDays = len(d[i])
    lastMonth = datetime.datetime.strptime(d[i][-1],'%d-%m-%y').month
    investmentAmount = (SIPamount*lastMonth)/totalDays
    for y in d[i]:
        z = list(etfdata['Date'].loc[lambda x:x==y].index)
        date = y
        price = etfdata.loc[z[0],'Close']
        qty = math.floor(investmentAmount/price)
        daySIP = daySIP.append(
                    pd.Series(
                        [
                            date,
                            datetime.datetime.strptime(date,'%d-%m-%y').weekday(),
                            price,
                            qty,
                            qty*price
                        ],index=my_columns
                    ),ignore_index=True
                )
daySIP.to_csv('Day SIP.csv')